In [1]:
## 규원 05.09 중앙일보 클롱링 csv파일 저장 
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv
from tqdm import tqdm  # tqdm 추가
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor, as_completed 
import time

## 2024년부터 오늘 날짜 까지 클롤링실시 
start_date = datetime(2023, 1, 1)
#end_date = datetime(2023, 1, 10)
end_date = datetime.now()

limit = 5

## 날짜 형식을 바꿔주는 함수 
def date_range(start, end):
    for n in range(int((end - start).days) + 1):
        yield start + timedelta(n)

# input : 해당 날짜의 url , output : 해당 날짜에 생성된 기사의 url List 
def fetch_news_articles(url):
    try:
        response = requests.get(url,timeout = limit)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        articles_url = []
        for article in soup.find_all('li', class_='card'):
            title_link = article.find('a')
            if title_link and title_link.has_attr('href'):
                articles_url.append(title_link['href'])
                #print(title_link['href'])
        return articles_url
    except requests.exceptions.Timeout:
        print("Timeout occurred while fetching articles from", url)
        return []
    except requests.exceptions.RequestException as e:
        print("Error occurred while fetching articles:", e)
        return []

## input : 기사의 url , output : 해당 기사에서 추출한 url, 제목, 내용의 튜플리스트
def fetch_articles_with_details(url):
    try:
        response = requests.get(url,timeout = limit)
        response.raise_for_status()  # HTTP 에러가 있을 경우 예외를 발생시킵니다.

        soup = BeautifulSoup(response.text, 'html.parser')

        articles = []

        for article in soup.find_all('article'):
            # 제목 추출
            headline = article.find('h1', class_='headline')
            title = headline.get_text(strip=True) if headline else "No title"
            # 본문 내용 추출
            body = article.find('div', class_='article_body fs3')
            text = []
            if body:
                for p in body.find_all('p', attrs={"data-divno": True}):
                    text.append(p.get_text(strip=True))
            text_content = ' '.join(text)

            print(url+"진행중")
            articles.append([url, title, text_content])

        return articles
    except requests.exceptions.Timeout:
        print("Timeout occurred while fetching articles from", url)
        return []
    except requests.exceptions.RequestException as e:
        print("Error occurred while fetching articles:", e)
        return []

# input : fetch_news_articles() 에 날짜를 for문으로 넣어서 사용
# output : 지정한 날짜의 모든 뉴스기사의 url list 
# 해당 작업은 그렇게 오래걸리지않음 5개월치 1분걸림 
def get_news_links():
    ## 시작 요일부터 하루씩 올려가며, 데이터 크롤링 
    links_url = []
    for single_date in date_range(start_date, end_date):
        formatted_date = single_date.strftime("%Y/%m/%d")
        # 날짜를 하루씩 증가시키면서, url 오픈 24월1월1일~ 
        url = f'https://www.joongang.co.kr/sitemap/index/{formatted_date}'
        # 해당 날짜에서의 기사 url list를 추출 
        links = fetch_news_articles(url)
        print(f"Date: {formatted_date}")
        for link in links:
            links_url.append(link)
    ## 모든 크롤링한 모든 링크의 url 전달, 1월1일부터의 ~ 모든 뉴스기사의 링크 
    return links_url


# 크롤링하고 결과를 바로 CSV 파일에 저장하는 함수
def joongang_crawler():
    # 결과를 저장할 폴더 생성
    result_dir = 'result'
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    # CSV 파일 경로 설정
    csv_file_path = os.path.join(result_dir, 'joongang_news.csv')
    
    # CSV 파일을 열고 데이터를 저장합니다.
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # CSV 파일의 헤더를 작성합니다.
        writer.writerow(['URL', 'Title', 'Content'])
            
        # 멀티쓰레드로 
        with ThreadPoolExecutor(max_workers=16) as executor:
            future_to_url = {}
            
            # 모든 날짜에 대해 URL을 생성하고 크롤링을 실행
            for single_date in date_range(start_date, end_date):
                formatted_date = single_date.strftime("%Y/%m/%d")
                url = f'https://www.joongang.co.kr/sitemap/index/{formatted_date}'
                
                # 해당 날짜의 기사 links 리스트 추출
                links = fetch_news_articles(url)
                #print(f"Date: {formatted_date} - Fetched {len(links)} articles")

                # 각 링크에 대해 비동기 작업을 예약합니다.
                for link in links:
                    future = executor.submit(fetch_articles_with_details, link)
                    future_to_url[future] = link

            # 완료된 작업을 순서대로 처리합니다.
            for future in as_completed(future_to_url):
                articles = future.result()
                if articles:
                    for article in articles:
                        writer.writerow(article)
                #print(f"Completed: {future_to_url[future]}")

# # CSV 파일 저장
if __name__ == "__main__":
    start_time = time.time()  # 코드 실행 시작 시간 기록
    joongang_crawler()
    end_time = time.time()  # 코드 실행 종료 시간 기록
    print(f"Execution time: {end_time - start_time} seconds")  # 실행 시간 출력

Timeout occurred while fetching articles from https://www.joongang.co.kr/article/25130607


KeyboardInterrupt: 